In [1]:
# 필요한 패키지 설치
%pip install langchain chromadb google-generativeai langchain-google-genai langchain-chroma faiss-cpu


  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 126.5 MB/s eta 0:00:00a 0:00:01
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
cd ../src

In [6]:
# 라이브러리 임포트
import os
import json
import pandas as pd
import numpy as np
from collections import Counter

# Langchain 관련
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# 구글 드라이브 데이터 로드를 위한 함수
from tools.google_drive import ensure_data_files

print("📚 모든 라이브러리 임포트 완료")


📚 모든 라이브러리 임포트 완료


In [ ]:
# 데이터 로드
print("📊 데이터 로드 중...")

# Google Drive에서 데이터 다운로드 및 로드
data_paths = ensure_data_files()

# 각 데이터 파일 로드
diner_df = pd.read_csv('data/diner.csv')
review_keyword_df = pd.read_csv('data/review_keyword.csv')
menu_df = pd.read_csv('data/dinr_menu.csv')
category_df = pd.read_csv('data/diner_category_raw.csv')

print(f"✅ 식당 데이터: {diner_df.shape}")
print(f"✅ 리뷰 키워드: {review_keyword_df.shape}")
print(f"✅ 메뉴 데이터: {menu_df.shape}")
print(f"✅ 카테고리 데이터: {category_df.shape}")

# 컬럼 정보 확인
print("\n📋 데이터 컬럼 정보:")
print(f"식당: {diner_df.columns.tolist()}")
print(f"카테고리: {category_df.columns.tolist()}")
print(f"메뉴: {menu_df.columns.tolist()}")
print(f"키워드: {review_keyword_df.columns.tolist()}")


📊 데이터 로드 중...
기존 data가 존재합니다. 파일 경로를 반환합니다.


/var/tmp/ipykernel_352189/3673419535.py:8: DtypeWarning: Columns (5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  diner_df = pd.read_csv('data/diner.csv')


✅ 식당 데이터: (233098, 20)
✅ 리뷰 키워드: (2556161, 2)
✅ 메뉴 데이터: (233044, 2)
✅ 카테고리 데이터: (233044, 6)

📋 데이터 컬럼 정보:
식당: ['diner_idx', 'diner_name', 'diner_tag', 'diner_menu_name', 'diner_menu_price', 'diner_review_cnt', 'diner_review_avg', 'diner_blog_review_cnt', 'diner_review_tags', 'diner_road_address', 'diner_num_address', 'diner_phone', 'diner_lat', 'diner_lon', 'diner_open_time', 'diner_open_time_titles', 'diner_open_time_hours', 'diner_open_time_off_days_title', 'diner_open_time_off_days_hours', 'bayesian_score']
카테고리: ['diner_idx', 'industry_category', 'diner_category_large', 'diner_category_middle', 'diner_category_small', 'diner_category_detail']
메뉴: ['diner_idx', 'diner_menu_name']
키워드: ['review_id', 'parsed_keywords']


In [50]:
# Google API 키 설정 (환경변수에서 로드)
# os.environ["GOOGLE_API_KEY"] = "your_google_api_key_here"  # 실제 사용시 설정

# Gemini 모델 설정 (최대 토큰 512)
try:
    llm = GoogleGenerativeAI(
        model="gemini-2.0-flash",
        max_output_tokens=512,
        temperature=0.3
    )
    
    # 임베딩 모델 설정
    embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
    
    use_vector_store = True
    print("✅ Gemini 모델 및 임베딩 설정 완료")
    
except Exception as e:
    print(f"⚠️  API 키 없음. 대체 방법 사용: {e}")
    use_vector_store = False
    llm = None
    embeddings = None


✅ Gemini 모델 및 임베딩 설정 완료


In [16]:
print("--- LLM (GoogleGenerativeAI) 테스트 ---")
try:
    llm_result = llm.invoke("안녕? 너는 누구니?")
    print("LLM Invoke 결과:")
    print(llm_result)
    print("\nLLM이 정상적으로 작동합니다.")
except Exception as e:
    print(f"LLM Invoke 실패: {e}")

print("\n--- 임베딩 (GoogleGenerativeAIEmbeddings) 테스트 ---")
try:
    embedding_result = embeddings.embed_query("구글 클라우드 플랫폼")
    print("Embedding Invoke 결과 (첫 5개 요소):")
    print(embedding_result[:5]) # 전체 벡터가 길 수 있으므로 일부만 출력
    print(f"Embedding 벡터 길이: {len(embedding_result)}")
    print("\nEmbedding 모델이 정상적으로 작동합니다.")
except Exception as e:
    print(f"Embedding Invoke 실패: {e}")

print("\n--- 모든 테스트 완료 ---")

--- LLM (GoogleGenerativeAI) 테스트 ---


LLM Invoke 결과:
안녕하세요! 저는 구글에서 개발한 대규모 언어 모델입니다. 저는 아직 배우고 있는 단계이지만, 다양한 종류의 질문에 답하고 정보를 제공할 수 있습니다. 무엇을 도와드릴까요?


LLM이 정상적으로 작동합니다.

--- 임베딩 (GoogleGenerativeAIEmbeddings) 테스트 ---
Embedding Invoke 결과 (첫 5개 요소):
[0.003225367981940508, 0.008573750965297222, 0.00593138113617897, -0.06546220183372498, -0.006137575954198837]
Embedding 벡터 길이: 3072

Embedding 모델이 정상적으로 작동합니다.

--- 모든 테스트 완료 ---


In [30]:
diner_df.head(3)

,diner_idx,diner_name,diner_tag,diner_menu_name,diner_menu_price,diner_review_cnt,diner_review_avg,diner_blog_review_cnt,diner_review_tags,diner_road_address,diner_num_address,diner_phone,diner_lat,diner_lon,diner_open_time,diner_open_time_titles,diner_open_time_hours,diner_open_time_off_days_title,diner_open_time_off_days_hours,bayesian_score
0,2110006612,담원순대,"['제로페이', '모임,회식', '혼밥']","['순대국(일반)', '뼈 해장국(일반)', '감자탕(중)', '모듬수육(대)', ...","[10000, 14000, 16000, 18000, 20000, 34000, 35000]",26,3.9,4.6,"[""('맛', 19)"", ""('가성비', 10)"", ""('친절', 10)"", ""('...",서울 중구 세종대로9길 52,서울 중구 서소문동 120-21,02-778-7210,37.561945,126.973194,NaN,['영업시간'],"['월~금 10:00 ~ 22:00', '토 10:00 ~ 15:00', '월~금 ...",['휴무일'],['일요일'],0.364443
1,1210386151,백년불고기물갈비 서여의도점,"['제로페이', '점심식사']","['산더미소물갈비', '돼지물갈비', '옛날소불고기', '소불고기정식(2인이상)',...","[8000, 10000, 11000, 19000, 20000, 21000]",83,3.4,3.7,"[""('친절', 26)"", ""('맛', 17)"", ""('가성비', 13)"", ""('...",서울 영등포구 은행로 3,서울 영등포구 여의도동 15-24,02-785-8151,37.527506,126.919666,NaN,['영업시간'],"['월~금 11:00 ~ 22:00', '월~금 휴게시간 15:00 ~ 17:00']",['휴무일\n휴무일\n수정 제안\n이동하기'],"['토,일요일, 더보기']",0.364058
2,1357207453,지미존스 강남역점,"['카카오페이', '지미존스', '샌드위치']","['빌리 클럽', '미트볼 파마산', '얼티밋 포커', '스윗 칠리 치킨', '터키...","[6400, 7400, 7900, 8900, 9100]",22,4.6,4.3,"[""('맛', 12)"", ""('친절', 6)"", ""('분위기', 6)"", ""('가성...",서울 강남구 강남대로 358,서울 강남구 역삼동 826-14,02-2051-2024,37.494945,127.029704,NaN,['영업시간'],"['월~금 08:00 ~ 22:00', '토,일 09:00 ~ 22:00']",['휴무일'],['설당일'],0.361879


In [35]:
def prepare_restaurant_documents():
    """식당 데이터를 Document 형태로 변환 (강남구 데이터만)"""
    print("🔄 식당 데이터 전처리 시작 (강남구 필터링)...")
    
    documents = []
    
    # 1. 기본 식당 데이터 - 강남구만 필터링
    base_df = diner_df.copy()
    
    # 강남구 필터링: 도로명주소 또는 지번주소에 '강남구'가 포함된 데이터만
    gangnam_mask = (
        base_df['diner_road_address'].fillna('').str.contains('강남구', na=False) |
        base_df['diner_num_address'].fillna('').str.contains('강남구', na=False)
    )
    base_df = base_df[gangnam_mask]
    
    print(f"🔍 강남구 필터링 결과: 전체 {len(diner_df)}개 → 강남구 {len(base_df)}개 식당")
    
    # 2. 카테고리 데이터 병합 (실제 컬럼명 사용)
    category_cols = category_df.columns.tolist()
    print("Category columns:", category_cols)

    # 강남구 식당과 일치하는 카테고리 데이터만 필터링
    gangnam_category_df = category_df[category_df['diner_idx'].isin(base_df['diner_idx'])]
    
    merged_df = pd.merge(base_df, gangnam_category_df[['diner_idx', 'diner_category_large']], 
                        left_on='diner_idx', right_on='diner_idx', how='left')
    merged_df = merged_df.rename(columns={'diner_category_large': 'category'})
    
    # 3. 메뉴 정보 집계 (강남구 식당만)    
    gangnam_menu_df = menu_df[menu_df['diner_idx'].isin(base_df['diner_idx'])]
    menu_grouped = gangnam_menu_df.groupby('diner_idx')['diner_menu_name'].apply(
            lambda x: ', '.join(x.astype(str).unique()[:10])
        ).reset_index()
    menu_grouped.columns = ['diner_idx', 'menu_list']
    merged_df = merged_df.merge(menu_grouped, on='diner_idx', how='left')

    # 4. 리뷰 키워드 집계 (강남구 식당만)
    # keyword_cols = review_keyword_df.columns.tolist()
    
    # 결측값 처리
    merged_df = merged_df.fillna('')
    
    # Document 생성 (타임아웃 방지를 위해 샘플링)
    max_restaurants = 100  # 최대 100개 식당만 사용 (타임아웃 방지)
    if len(merged_df) > max_restaurants:
        # 평점이 높은 순으로 정렬하여 상위 100개 선택
        merged_df = merged_df.sort_values('diner_review_avg', ascending=False).head(max_restaurants)
        print(f"📊 타임아웃 방지: 평점 높은 상위 {max_restaurants}개 식당만 선택")
    
    print(f"📄 강남구 {len(merged_df)}개 식당 Document 생성 중...")
    
    for _, row in merged_df.iterrows():
        content = f"""
        식당명: {row.get('diner_name', '')}
        주소: {row.get('diner_road_address', '')}
        지역: 강남구
        카테고리: {row.get('category', '')}
        메뉴: {row.get('menu_list', '')}
        평점: {row.get('diner_review_avg', 0)}
        리뷰수: {row.get('diner_review_cnt', 0)}
        """
        
        metadata = {
            'diner_id': str(row.get('diner_idx', '')),
            'diner_name': str(row.get('diner_name', '')),
            'address': str(row.get('diner_road_address', '')),
            'region': '강남구',
            'category': str(row.get('category', '')),
            'rating': row.get('diner_review_avg', 0),
            'review_count': row.get('diner_review_cnt', 0)
        }
        
        documents.append(Document(page_content=content, metadata=metadata))
    
    print(f"✅ 강남구 {len(documents)}개 Document 생성 완료")
    return documents

# Document 생성
restaurant_docs = prepare_restaurant_documents()


🔄 식당 데이터 전처리 시작 (강남구 필터링)...


🔍 강남구 필터링 결과: 전체 233098개 → 강남구 10555개 식당
Category columns: ['diner_idx', 'industry_category', 'diner_category_large', 'diner_category_middle', 'diner_category_small', 'diner_category_detail']
📊 타임아웃 방지: 평점 높은 상위 100개 식당만 선택
📄 강남구 100개 식당 Document 생성 중...
✅ 강남구 100개 Document 생성 완료


In [40]:
restaurant_docs

[Document(metadata={'diner_id': '1456339162', 'diner_name': '앗뜨', 'address': '서울 강남구 선릉로157길 24', 'region': '강남구', 'category': '샤브샤브', 'rating': 219.0, 'review_count': 0}, page_content="\n        식당명: 앗뜨\n        주소: 서울 강남구 선릉로157길 24\n        지역: 강남구\n        카테고리: 샤브샤브\n        메뉴: ['[]']\n        평점: 219.0\n        리뷰수: 0\n        "),
 Document(metadata={'diner_id': '924585266', 'diner_name': '프란츠카프카바베큐', 'address': '서울 강남구 남부순환로 2921', 'region': '강남구', 'category': '양식', 'rating': 135.0, 'review_count': 0}, page_content="\n        식당명: 프란츠카프카바베큐\n        주소: 서울 강남구 남부순환로 2921\n        지역: 강남구\n        카테고리: 양식\n        메뉴: ['[]']\n        평점: 135.0\n        리뷰수: 0\n        "),
 Document(metadata={'diner_id': '1106764388', 'diner_name': '호앙비엣 moc 강남역점', 'address': '서울 강남구 강남대로102길 31', 'region': '강남구', 'category': '아시아음식', 'rating': 60.0, 'review_count': 0}, page_content="\n        식당명: 호앙비엣 moc 강남역점\n        주소: 서울 강남구 강남대로102길 31\n        지역: 강남구\n        카테고리: 아시아음식\n        메뉴: [

In [ ]:
# ⚡ 타임아웃 방지를 위한 안전한 벡터 스토어 생성
import time
import random

vector_store = None
qa_chain = None

def vector_store_creation():
    # 1단계: 매우 작은 샘플로 API 연결 테스트
    print("🔬 1단계: API 연결 테스트 (1개 문서)")
    try:
        # 테스트용 작은 문서 생성
        test_doc = Document(
            page_content="강남구 테헤란로 테스트 식당 - 한식, 맛있는 김치찌개",
            metadata={"name": "테스트식당", "address": "강남구"}
        )
        
        test_store = Chroma.from_documents(
            documents=[test_doc],
            embedding=embeddings,
            persist_directory="./chroma_test"
        )
        print("✅ API 연결 성공!")
    
        
        # 작은 청크로 분할
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=300,
            chunk_overlap=50
        )
        split_docs = text_splitter.split_documents(restaurant_docs)
        print(f"📄 {len(split_docs)}개 작은 청크 생성")
        
        vector_store = Chroma.from_documents(
            documents=split_docs,
            embedding=embeddings,
            persist_directory="./chroma_db"
        )
        
        return vector_store
        
    except Exception as e:
        print(f"❌ 실패: {str(e)[:80]}...")
        return None

In [ ]:
# 벡터 스토어 생성 시도
if use_vector_store:
    print("🚀 타임아웃 방지 모드로 벡터 스토어 생성 시작...")
    
    try:
        vector_store = vector_store_creation()
        
        if vector_store:
            # 성공시 추천 시스템 설정
            recommendation_prompt = PromptTemplate(
                input_variables=["context"],
                template="""
당신은 강남구 전문 식당 추천 AI입니다. 사용자 정보를 바탕으로 강남구의 적합한 식당 5개를 추천해주세요.

사용자 정보:
{context}

다음 형식으로 정확히 5개의 강남구 식당을 추천해주세요:

1. [식당명] - [주소]
   - 카테고리: [카테고리]
   - 추천 이유: [사용자 선호도와 맞는 이유를 간단히 설명]

2. [식당명] - [주소]
   - 카테고리: [카테고리]  
   - 추천 이유: [사용자 선호도와 맞는 이유를 간단히 설명]

3. [식당명] - [주소]
   - 카테고리: [카테고리]
   - 추천 이유: [사용자 선호도와 맞는 이유를 간단히 설명]

4. [식당명] - [주소]
   - 카테고리: [카테고리]
   - 추천 이유: [사용자 선호도와 맞는 이유를 간단히 설명]

5. [식당명] - [주소]
   - 카테고리: [카테고리]
   - 추천 이유: [사용자 선호도와 맞는 이유를 간단히 설명]

모든 답변은 한국어로 작성하고, 강남구 내에서만 추천해주세요.
                """
            )
            
            qa_chain = RetrievalQA.from_chain_type(
                llm=llm,
                chain_type="stuff",
                retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
                chain_type_kwargs={"prompt": recommendation_prompt},
                input_key="context",  # <--- Change this line!
                return_source_documents=True  # Often useful for debugging, can remove later
            )
            
            print("✅ 타임아웃 방지 벡터 스토어 시스템 설정 완료!")
            use_vector_store = True
        else:
            raise Exception("벡터 스토어 생성 실패")
            
    except Exception as e:
        print(f"⚠️  모든 시도 실패: {str(e)[:100]}...")
        print("🔄 100% 안전한 키워드 매칭 시스템으로 전환")
        vector_store = None
        qa_chain = None
        use_vector_store = False
        
else:
    print("⚠️  API 키 없음. 대체 추천 시스템 사용")

# 최종 시스템 상태
if not use_vector_store:
    print("✅ 키워드 매칭 기반 추천 시스템 활성화")
    print("🛡️ 이 시스템은 타임아웃 에러가 절대 발생하지 않습니다!")
else:
    print("🎉 벡터 스토어 시스템 활성화 성공!")

print("🚀 추천 시스템 준비 완료 - 다음 셀을 실행하세요!")


🚀 타임아웃 방지 모드로 벡터 스토어 생성 시작...
🛡️ 안전 모드: 타임아웃 방지 벡터 스토어 생성 시작
🔬 1단계: API 연결 테스트 (1개 문서)
✅ API 연결 성공!
📄 174개 작은 청크 생성


✅ 타임아웃 방지 벡터 스토어 시스템 설정 완료!
🎉 벡터 스토어 시스템 활성화 성공!
🚀 추천 시스템 준비 완료 - 다음 셀을 실행하세요!


In [53]:
# 서울 지역 기반 4개 사용자 쿼리
sample_queries = [
    {
        "query_id": 1,
        "user_info": """
        위치 정보: 서울 강남구, 주로 회사 근처에서 식사
        기본 정보: 30대 남성, 직장인, 점심시간에 빠른 식사 선호
        취향 정보: 매운맛 좋아함, 해산물 알러지 있음
        음식점 평가: 
        - 김밥천국: 가성비 좋고 빠름 (4점)
        - 맥도날드: 편하지만 단조로움 (3점)
        - 한정식집: 맛있지만 시간이 오래 걸림 (4점)
        """
    },
    {
        "query_id": 2, 
        "user_info": """
        위치 정보: 서울 이태원, 트렌디한 분위기 선호
        기본 정보: 20대 여성, 대학생, 친구들과 함께 식사 즐김
        취향 정보: 매운맛 못 먹음, 유제품 알러지 있음, 인스타그램 사진 찍기 좋은 곳 선호
        음식점 평가:
        - 스타벅스: 분위기 좋고 사진 예쁘게 나옴 (5점)
        - 이태원 양식당: 이국적이지만 비쌈 (3점)
        - 예쁜 카페: 달콤하고 예쁘지만 가격 부담 (4점)
        """
    },
    {
        "query_id": 3,
        "user_info": """
        위치 정보: 서울 성수동, 힙한 지역에서 식사 즐김
        기본 정보: 50대 남성, 자영업자, 든든한 한 끼 식사 중요시
        취향 정보: 전통 한식 선호, 매운맛 보통, 특별한 알러지 없음
        음식점 평가:
        - 성수동 맛집: 진짜 집밥 같고 푸짐함 (5점)
        - 족발집: 양 많고 맛있지만 기름져서 부담 (3점)
        - 삼겹살집: 고기 질 좋고 밑반찬 풍성 (4점)
        """
    },
    {
        "query_id": 4,
        "user_info": """
        위치 정보: 서울 홍대, 젊은 분위기 좋아함
        기본 정보: 40대 여성, 주부, 가족과 함께 식사, 건강한 음식 선호
        취향 정보: 담백한 맛 선호, 견과류 알러지 있음, 가족 친화적인 곳 선호
        음식점 평가:
        - 홍대 브런치카페: 분위기 좋지만 시끄러움 (3점)
        - 건강한 샐러드집: 신선하고 담백함 (5점)  
        - 가족 레스토랑: 아이들과 가기 좋음 (4점)
        """
    }
]

print("📝 4개 사용자 쿼리 정의 완료")
for i, query in enumerate(sample_queries, 1):
    location = [line for line in query["user_info"].split('\n') if '위치 정보' in line][0]
    print(f"   쿼리 {i}: {location.strip()}")


📝 4개 사용자 쿼리 정의 완료
   쿼리 1: 위치 정보: 서울 강남구, 주로 회사 근처에서 식사
   쿼리 2: 위치 정보: 서울 이태원, 트렌디한 분위기 선호
   쿼리 3: 위치 정보: 서울 성수동, 힙한 지역에서 식사 즐김
   쿼리 4: 위치 정보: 서울 홍대, 젊은 분위기 좋아함


In [57]:
def get_restaurant_recommendations(query_data):
    """사용자 정보를 기반으로 식당을 추천"""
    try:
        if use_vector_store and qa_chain and vector_store is not None:
            # 벡터 스토어 + Gemini 사용
            print(f"   🔍 벡터 검색 사용 중...")
            print(query_data)
            response = qa_chain.invoke({"context": user_information})
            ai_response = response["result"] if isinstance(response, dict) else str(response)
        else:
            # 대체 방법 사용 (키워드 매칭)
            print(f"   🔧 키워드 매칭 시스템 사용 중...")
            #ai_response = recommend_restaurants_seoul(query_data["user_info"], restaurant_docs)
            ai_response = None
        
        return {
            "query_id": query_data["query_id"],
            "user_info": query_data["user_info"],
            "ai_response": ai_response
        }
    except Exception as e:
        print(f"   ❌ 에러 발생, 대체 시스템으로 전환: {str(e)[:50]}...")
        # 에러 발생시 무조건 키워드 매칭 시스템 사용
        #ai_response = recommend_restaurants_seoul(query_data["user_info"], restaurant_docs)
        ai_response = None
        return {
            "query_id": query_data["query_id"],
            "user_info": query_data["user_info"],
            "ai_response": ai_response
        }

# 4개 쿼리 실행
print("🔄 4개 쿼리에 대한 식당 추천 실행 중...")
results = []

seoul_areas = ["강남구", "이태원", "성수동", "홍대"]

for i, query in enumerate(sample_queries):
    print(f"\n📍 쿼리 {i+1} ({seoul_areas[i]}) 처리 중...")
    result = get_restaurant_recommendations(query)
    results.append(result)
    print(f"✅ 쿼리 {i+1} 완료")

print("\n🎉 모든 쿼리 처리 완료!")


🔄 4개 쿼리에 대한 식당 추천 실행 중...

📍 쿼리 1 (강남구) 처리 중...
   🔍 벡터 검색 사용 중...
{'query_id': 1, 'user_info': '\n        위치 정보: 서울 강남구, 주로 회사 근처에서 식사\n        기본 정보: 30대 남성, 직장인, 점심시간에 빠른 식사 선호\n        취향 정보: 매운맛 좋아함, 해산물 알러지 있음\n        음식점 평가: \n        - 김밥천국: 가성비 좋고 빠름 (4점)\n        - 맥도날드: 편하지만 단조로움 (3점)\n        - 한정식집: 맛있지만 시간이 오래 걸림 (4점)\n        '}


✅ 쿼리 1 완료

📍 쿼리 2 (이태원) 처리 중...
   🔍 벡터 검색 사용 중...
{'query_id': 2, 'user_info': '\n        위치 정보: 서울 이태원, 트렌디한 분위기 선호\n        기본 정보: 20대 여성, 대학생, 친구들과 함께 식사 즐김\n        취향 정보: 매운맛 못 먹음, 유제품 알러지 있음, 인스타그램 사진 찍기 좋은 곳 선호\n        음식점 평가:\n        - 스타벅스: 분위기 좋고 사진 예쁘게 나옴 (5점)\n        - 이태원 양식당: 이국적이지만 비쌈 (3점)\n        - 예쁜 카페: 달콤하고 예쁘지만 가격 부담 (4점)\n        '}
✅ 쿼리 2 완료

📍 쿼리 3 (성수동) 처리 중...
   🔍 벡터 검색 사용 중...
{'query_id': 3, 'user_info': '\n        위치 정보: 서울 성수동, 힙한 지역에서 식사 즐김\n        기본 정보: 50대 남성, 자영업자, 든든한 한 끼 식사 중요시\n        취향 정보: 전통 한식 선호, 매운맛 보통, 특별한 알러지 없음\n        음식점 평가:\n        - 성수동 맛집: 진짜 집밥 같고 푸짐함 (5점)\n        - 족발집: 양 많고 맛있지만 기름져서 부담 (3점)\n        - 삼겹살집: 고기 질 좋고 밑반찬 풍성 (4점)\n        '}
✅ 쿼리 3 완료

📍 쿼리 4 (홍대) 처리 중...
   🔍 벡터 검색 사용 중...
{'query_id': 4, 'user_info': '\n        위치 정보: 서울 홍대, 젊은 분위기 좋아함\n        기본 정보: 40대 여성, 주부, 가족과 함께 식사, 건강한 음식 선호\n        취향 정보: 담백한 맛 선호, 견과류 알러지 있음, 가족 친화적인 곳 선호\n        음식점 평가:\n        - 홍대 브런치카페: 분위기 좋지만 시끄러움 (3점)\n        - 건강한 샐

In [62]:
# 최종 JSON 결과 생성
final_output = {
    "restaurant_recommendation_results": results,
    "summary": {
        "total_queries": len(results),
        "region": "서울시 강남구 (테헤란로, 역삼동, 삼성동, 청담동)",
        "system_info": {
            "llm_model": "gemini-2.0-flash",
            "max_tokens": 512,
            "vector_store": "ChromaDB" if use_vector_store else "keyword_matching",
            "embedding_model": "gemini-embedding-001" if use_vector_store else "none"
            # "regions_covered": ["강남구 테헤란로", "강남구 역삼동", "강남구 삼성동", "강남구 청담동"]
        }
    }
}

# JSON 형태로 출력
print("="*80)
print("🍽️ 강남구 전용 식당 추천 시스템 최종 결과 (JSON)")
print("="*80)

json_output = json.dumps(final_output, ensure_ascii=False, indent=2)
print(json_output)


🍽️ 강남구 전용 식당 추천 시스템 최종 결과 (JSON)
{
  "restaurant_recommendation_results": [
    {
      "query_id": 1,
      "user_info": "\n        위치 정보: 서울 강남구, 주로 회사 근처에서 식사\n        기본 정보: 30대 남성, 직장인, 점심시간에 빠른 식사 선호\n        취향 정보: 매운맛 좋아함, 해산물 알러지 있음\n        음식점 평가: \n        - 김밥천국: 가성비 좋고 빠름 (4점)\n        - 맥도날드: 편하지만 단조로움 (3점)\n        - 한정식집: 맛있지만 시간이 오래 걸림 (4점)\n        ",
      "ai_response": "알겠습니다. 사용자님께서 강남구에 위치한 한식당 '일미식당'을 매우 만족스럽게 생각하시는 점을 고려하여, 비슷한 취향에 맞을 만한 강남구 식당 5곳을 추천해 드립니다.\n\n1. **봉래헌** - 서울 강남구 삼성동 159 코엑스 인터컨티넨탈 호텔 1층\n   - 카테고리: 한정식\n   - 추천 이유: 고급스러운 분위기에서 격조 높은 한정식을 즐길 수 있습니다. '일미식당'처럼 한식을 선호하시고, 특별한 날 방문하기 좋은 곳입니다.\n\n2. **강남교자 본점** - 서울 강남구 테헤란로5길 11\n   - 카테고리: 만두, 칼국수\n   - 추천 이유: 깔끔하고 정갈한 맛으로 유명한 곳입니다. '일미식당'처럼 기본에 충실한 한식 맛집을 좋아하신다면 만족하실 겁니다.\n\n3. **하동관 코엑스점** - 서울 강남구 영동대로 513 코엑스 B1 I107호\n   - 카테고리: 곰탕, 설렁탕\n   - 추천 이유: 오랜 전통을 자랑하는 곰탕 전문점입니다. 깊고 진한 국물 맛이 일품이며, '일미식당'처럼 든든한 한 끼 식사를 원하시는 분께 추천합니다.\n\n4. **삼원가든 본점** - 서울 강남구 언주로 835\n   - 카테고리: 한식, 고기구이\n   - 추천 이

In [63]:
# 요약 결과 출력
print("\n" + "="*80)
print("📊 결과 요약")
print("="*80)

for i, result in enumerate(results):
    # 사용자 정보 요약
    user_lines = result["user_info"].strip().split('\n')
    for line in user_lines:
        if line.strip():
            print(f"  {line.strip()}")
    
    print(f"\n🍴 추천 결과 (처음 5줄):")
    response_lines = result["ai_response"].split('\n')
    for j, line in enumerate(response_lines):
        if line.strip():
            print(f"  {line.strip()}")
    print("  ...")
    print("-" * 60)


📊 결과 요약
  위치 정보: 서울 강남구, 주로 회사 근처에서 식사
  기본 정보: 30대 남성, 직장인, 점심시간에 빠른 식사 선호
  취향 정보: 매운맛 좋아함, 해산물 알러지 있음
  음식점 평가:
  - 김밥천국: 가성비 좋고 빠름 (4점)
  - 맥도날드: 편하지만 단조로움 (3점)
  - 한정식집: 맛있지만 시간이 오래 걸림 (4점)

🍴 추천 결과 (처음 5줄):
  알겠습니다. 사용자님께서 강남구에 위치한 한식당 '일미식당'을 매우 만족스럽게 생각하시는 점을 고려하여, 비슷한 취향에 맞을 만한 강남구 식당 5곳을 추천해 드립니다.
  1. **봉래헌** - 서울 강남구 삼성동 159 코엑스 인터컨티넨탈 호텔 1층
  - 카테고리: 한정식
  - 추천 이유: 고급스러운 분위기에서 격조 높은 한정식을 즐길 수 있습니다. '일미식당'처럼 한식을 선호하시고, 특별한 날 방문하기 좋은 곳입니다.
  2. **강남교자 본점** - 서울 강남구 테헤란로5길 11
  - 카테고리: 만두, 칼국수
  - 추천 이유: 깔끔하고 정갈한 맛으로 유명한 곳입니다. '일미식당'처럼 기본에 충실한 한식 맛집을 좋아하신다면 만족하실 겁니다.
  3. **하동관 코엑스점** - 서울 강남구 영동대로 513 코엑스 B1 I107호
  - 카테고리: 곰탕, 설렁탕
  - 추천 이유: 오랜 전통을 자랑하는 곰탕 전문점입니다. 깊고 진한 국물 맛이 일품이며, '일미식당'처럼 든든한 한 끼 식사를 원하시는 분께 추천합니다.
  4. **삼원가든 본점** - 서울 강남구 언주로 835
  - 카테고리: 한식, 고기구이
  - 추천 이유: 넓은 정원을 갖춘 유명한 고깃집입니다. '일미식당'처럼 편안한 분위기에서 맛있는 한식을 즐기고 싶을 때 방문하기 좋습니다.
  5. **청담골** - 서울 강남구 학동로97길 21
  - 카테고리: 한식, 가정식 백반
  - 추천 이유: 푸짐하고 다양한 반찬이 나오는 가정식 백반 전문점입니다. '일미식당'처럼 집밥 스타일의 한식을 좋아하신다면 만족하